In [2]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2
    
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof
                 
if __name__ == "__main__":
    pop, stats, hof = main()

gen	nevals	avg                          	std                        	min                          	max                          
0  	50    	[  19.52        231.98214002]	[  6.58252231  69.12192829]	[   6.          100.29201362]	[  36.          408.42409383]
1  	93    	[   5.94        130.56782075]	[   7.16215052  131.43022643]	[ 0.  0.]                    	[  22.          408.42409383]
2  	85    	[  2.42        70.83298161]  	[   5.37806657  122.97049141]	[ 0.  0.]                    	[  22.          417.53104832]
3  	90    	[  1.96        83.73195692]  	[   3.42321486  110.37429143]	[ 0.  0.]                    	[  15.          430.52070247]
4  	85    	[   2.66        129.25469971]	[   3.8137121  110.659008 ]  	[ 0.  0.]                    	[  15.          430.52070247]
5  	88    	[   1.78        119.23923769]	[  2.72242539  77.43934674]  	[ 0.  0.]                    	[  15.          430.52070247]
6  	90    	[   1.96        124.03438261]	[  2.95269369  83.22943016]  	[ 0.  0.]       

In [5]:
print hof

[Individual([]), Individual([2]), Individual([8, 2]), Individual([8, 2, 18]), Individual([8, 2, 6]), Individual([8, 2, 18, 6]), Individual([8, 2, 12, 6]), Individual([8, 2, 12, 18, 6]), Individual([8, 2, 12, 6, 14]), Individual([2, 6, 8, 12, 14, 18]), Individual([1, 2, 6, 8, 12, 18]), Individual([1, 2, 6, 8, 12, 14]), Individual([2, 6, 7, 8, 12, 14, 18]), Individual([2, 6, 7, 8, 9, 12, 14, 18]), Individual([2, 6, 7, 8, 9, 10, 12, 14, 18]), Individual([1, 2, 6, 7, 8, 12, 14]), Individual([1, 2, 6, 7, 8, 9, 12, 14]), Individual([0, 2, 6, 7, 8, 9, 12, 14, 18]), Individual([2, 3, 6, 7, 8, 12, 14, 18]), Individual([1, 2, 3, 6, 7, 8, 12, 18]), Individual([1, 2, 3, 6, 7, 8, 10, 12, 18]), Individual([1, 2, 3, 6, 7, 8, 12, 14, 18]), Individual([1, 2, 3, 6, 7, 8, 10, 12, 14, 18]), Individual([1, 2, 3, 6, 7, 8, 12, 14, 16, 18]), Individual([1, 2, 3, 6, 7, 8, 9, 12, 14, 16, 18]), Individual([1, 2, 3, 5, 6, 7, 8, 12, 14, 18]), Individual([1, 2, 3, 5, 6, 7, 8, 12, 14, 16, 18]), Individual([1, 2, 3, 